# Analyzing Delays on Caltrain
by Özge Terzioğlu
14 February 2023

This is a continuation of my "Exploring Delays on Caltrain" notebook, where I documented my process of collecting, filtering, and merging the tweets from @Caltrain and @CaltrainAlerts about train delay on Twitter. 

In this notebook, we'll be analyzing Caltrain delays via the tweets and documenting preliminary data visualizations about this dataset. 

## Let's import our tools

In [1]:
import csv
import pandas as pd 
import re
import altair as alt
import calendar

### Read the local file into this notebook to be analyzed. 

In [3]:
cwd = (r"/Users/ozgeterzioglu/tracking-delays-with-caltrain-tweets/data/raw/complete_data.csv")
passenger_file = (r"/Users/ozgeterzioglu/tracking-delays-with-caltrain-tweets/data/analysis/passenger_data.csv")
tweets = pd.read_csv(cwd)
passenger_data = pd.read_csv(passenger_file)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ozgeterzioglu/tracking-delays-with-caltrain-tweets/data/processed/complete_data.csv'

## Next, we'll split the date column into day, month, year, and hour of the day for analysis 

### Change the date and time columns to a datetime object so we can split it into day, month, year, and hour. 

In [ ]:
tweets["created_at"] = pd.to_datetime(tweets["created_at"], format = "%m/%d/%y")

In [ ]:
tweets["time"] = pd.to_datetime(tweets["time"], format = "%H:%M:%S")

### Insert new columns for day of week, month, year, and hour. Assign the corresponding data into each column.

In [ ]:
tweets.insert(11, "day_of_week", "")

In [ ]:
tweets["day_of_week"] = tweets["created_at"].dt.day_name()

In [ ]:
tweets.insert(7, "day","")

In [ ]:
tweets["day"] = tweets["created_at"].dt.day

In [ ]:
tweets.insert(8, "month","")

In [ ]:
tweets["month"] = tweets["created_at"].dt.month

In [ ]:
tweets.insert(9, "year","")

In [ ]:
tweets["year"] = tweets["created_at"].dt.year

In [ ]:
tweets.insert(10, "hour","")

In [ ]:
tweets["hour"] = tweets["time"].dt.hour

In [ ]:
date = tweets["created_at"]

## What does our dataframe look like now?

In [ ]:
tweets.info()

In [ ]:
tweets.head()

### Let's change the year data type to string to make analysis easier.

In [ ]:
tweets = tweets.astype({"year": "string"})

### Now we'll filter our data frame to exclude 2023 (since we only have about a month's worth of data from this year).

In [ ]:
tweets_2012_2022 = tweets[(tweets.year >= "2012") & (tweets.year <= "2022")]
tweets_2012_2022.info()

### Let's clean out some extraneous/wrong station names to make our analysis accurate. 

In [ ]:
tweets_2012_2022 = tweets_2012_2022.astype({"station": "string"})

In [ ]:
include = r"\bSFK\b|\bTWE\b|\bBAY\b|\bSSF\b|\bSBR\b|\bMIL\b|\bBWY\b|\bBUR\b|\bSMT\b|\bHPK\b|\bHIL\b|\bBEL\b|\bSCS\b|\bRWC\b|\bMPK\b|\bPAL\b|\bSTF\b|\bCAL\b|\bSAT\b|\bMVW\b|\bSUN\b|\bLAW\b|\bSCL\b|\bCPK\b|\bSJD\b|\bTAM\b|\bCAP\b|\bBHL\b|\bMHL\b|\bSMR\b|\bGIL\b"
exclude = r"@"
include_mask = tweets_2012_2022["station"].str.contains(include)
exclude_mask = tweets_2012_2022["station"].str.startswith(exclude)
tweets_2012_2022 = tweets_2012_2022[include_mask & ~exclude_mask]

In [ ]:
tweets_2012_2022

Much better!

### Are there any empty rows in the minutes delayed column?

In [ ]:
print(tweets[pd.isnull(tweets["clean_delay"])])

Hmmm... it looks like there's about 300 empty rows in the clean_delay row, which is the length of the reported train delay in minutes. It was inevitable to run into some issues filtering tweets and pulling the text from them with regular expressions. We know that I wasn't able to extract delays longer than 2 digits (e.g. 100 minutes delayed). These were pretty rare in the dataset, but still important. I reckon some of these empty rows were triple digit delays. Due to lack of time to fix this further, we will ignore these empty rows for now **(but for an accurate report, the filter.py script should be debugged to catch three digit delay lengths).**

## Let's get started with some basic analysis! 

### We'll make a few .csv files so I can import them to Tableau or Flourish to create more polished visuals for publication.

We want to see on a large scale, the average number of delays per year:

In [ ]:
delays_per_year = tweets_2012_2022.groupby("year")["clean_delay"].count().reset_index()

In [ ]:
alt.Chart(delays_per_year).mark_bar().encode(
    x = "year",
    y = "clean_delay",
    color = "clean_delay"
).properties(
    title = "Number of Caltrain Delays Per Year",
    width = 600,
    height = 500
)

Seems like the number of delays peaked in 2019, right before the pandemic started. Delays were manually tweeted from 2012-2022. We should ask why this is. We can assume that delays dropped off in 2020 because service was reduced or cancelled due to the pandemic. 

per route:

In [ ]:
delays_by_route = tweets_2012_2022.groupby("clean_route")["clean_delay"].mean().reset_index()

In [ ]:
delays_by_route.to_csv("avg_delays_by_route.csv")

In [ ]:
alt.Chart(delays_by_route).mark_bar().encode(
    x = "clean_route",
    y = "clean_delay",
    color = "clean_delay"
).properties(
    title = "Average Caltrain Delays Per Route",
    width = 800,
    height = 500
)

and its median counterpart in case average is wonky due to outliers:

In [ ]:
delays_by_route_median = tweets_2012_2022.groupby("clean_route")["clean_delay"].median().reset_index()

In [ ]:
delays_by_route_median.to_csv("delays_by_route.csv")

In [ ]:
alt.Chart(delays_by_route_median).mark_bar().encode(
    x = "clean_route",
    y = "clean_delay",
    color = "clean_delay"
).properties(
    title = "Median Number of Caltrain Delays Per Year",
    width = 800,
    height = 500
)

and lastly, the average number of delays by each unique day:

In [ ]:
delays_by_date = tweets_2012_2022.groupby("created_at")["clean_delay"].mean().reset_index().set_axis(["date", "minutes_delayed"], axis = 1)
delays_by_date

In [ ]:
delays_by_date.to_csv("delays_by_day.csv")

In [ ]:
alt.Chart(delays_by_date).mark_bar().encode(
    x = "date",
    y = "minutes_delayed",
    color = "minutes_delayed"
).properties(
    title = "Average Number of Caltrain Delays Per Year",
    width = 800,
    height = 500
)

It's going to be hard quantifying the different number of reasons per year since the way the reasons were written is not standardized, but it might be useful to have in case we find the time to standardize the reasons and produce and analysis.

In [ ]:
reason_by_year = tweets_2012_2022.groupby("year")["delay_reason"].value_counts()
reason_by_year

In [ ]:
reason_by_year.to_csv("reason_by_year.csv")

total number of delays per station:

In [ ]:
count_delay_per_station = tweets_2012_2022.groupby("station")["clean_delay"].count().reset_index().set_axis(["station", "delays_count"], axis = 1)

In [ ]:
count_delay_per_station.to_csv("num_delays_by_station.csv")

In [ ]:
alt.Chart(count_delay_per_station).mark_bar().encode(
    x = "station",
    y = "delays_count",
    color = "delays_count"
).properties(
    title = "Number of Train Delays Per Station",
    width = 500,
    height = 200
)

Looks like there's very little to no data for some stations. This is a key point we should ask the Caltrains spokesperson about. I know, for instance, that stations like GIL (Gilroy) there is limited service there to begin with, so delays would naturally be lower.

## Let's organize the number of delays by the time of day

In [ ]:
grouped_station = tweets_2012_2022.groupby(["station", "created_at","hour"])

In [ ]:
grouped_hour = tweets_2012_2022.groupby(["created_at","hour"])

In [ ]:
with open("stationdelays_bytime.csv", mode = "w") as file:
    writer = csv.writer(file, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
    writer.writerow(["station","date","hour","avg_delay"])
    for station_key, station_item in grouped_station:
        grouped_delay = grouped_station.get_group(station_key)["clean_delay"].median()
        station_name = station_key[0]
        date = station_key[1].strftime("%Y-%m-%d")
        hour = station_key[2]
        writer.writerow([station_name, date, hour, grouped_delay])

## Since 2012, how many tweets provided context or reasoning for the delay? 

In [ ]:
reason_count = tweets_2012_2022["delay_reason"].count()

In [ ]:
tweets_2012_2022.count()

In [ ]:
total = 10420

In [ ]:
reason_count / total

This is something to ask Caltrain about, why are commuters notified of the cause of the train delay only 5% of the time?

## How many passengers were affected by delays on Caltrain from 2016-2019?

Let's see what we're working with.

In [ ]:
passenger_data.info()

Change the column names to snake case for smooth analysis and change the data type for `year` from integer to string so it's displayed properly. 

In [ ]:
passenger_data.columns = [col.lower().replace(" ","_") for col in passenger_data.columns]
passenger_data = passenger_data.astype({"year": "string"})
passenger_data

Looks like we're working with a dataset that spans from 2012-2019. Since 2012-2015 doesn't have data for the "On Board" column, which is the column we want to look at when quantifying how many passengers were affected by delays, we will filter out these years and do a short-term analysis instead. I know this data is missing because I created this dataset myself since I had to pull from multiple pdf files and spreadsheets to create it. 

We'll also have to edit the station names to match the tweets dataframe. We'll do this because we're going to merge these two dataframes together soon and create some viz with it.

In [ ]:
filtered_passengers = passenger_data[(passenger_data.year >= "2016") & (passenger_data.year <= "2019")]
filtered_passengers = filtered_passengers.replace({
    'San Francisco' : 'SFK',
    '22nd Street' : 'TWE',
    "Bayshore" : "BAY",
    "South San Francisco" : "SSF",
    "San Bruno" : "SBR",
    "Millbrae" : "MIL",
    "Broadway" : "BWY",
    "Burlingame" : "BUR",
    "San Mateo" : "SMT",
    "Hayward Park" : "HPK",
    "Hillsdale" : "HIL",
    "Belmont" : "BEL",
    "San Carlos" : "SCS",
    "Redwood City" : "RWC",
    "Atherton" : "ATH",
    "Menlo Park" : "MPK",
    "Palo Alto" : "PAL",
    "California Avenue" : "CAL",
    "San Antonio" : "SAT",
    "Mountain View" : "MVW",
    "Sunnyvale" : "SUN",
    "Lawrence" : "LAW",
    "Santa Clara" : "SCL",
    "College Park" : "CPK",
    "San Jose Diridon" : "SJD",
    "Tamien" : "TAM",
    "Capitol" : "CAP",
    "Blossom Hill" : "BHL",
    "Morgan Hill" : "MHL",
    "San Martin" : "SMR",
    "Gilroy" : "GIL"
    
})
filtered_passengers

In [ ]:
alt.Chart(passengers_onboard).mark_line().encode(
    x = "year",
    y = "on_board"
).properties(
    title = "Average Number of On Board Passengers Riding Caltrain from All Stations",
    width = 300,
    height = 200
)

The number of passengers riding Caltrain seems pretty stable between 2016-2019 and even on a slight incline. It would be more useful if we could see how ridership is affected by the pandemic. 

Let's compare the ridership from each station from 2016-2019 with delays.

We'll first filter our delays dataset for 2016-2019 to match the passenger dataframe. 

In [ ]:
tweets_2016_2019 = tweets[(tweets.year >= "2016") & (tweets.year <= "2019")]

Then we'll clean up our station names again.

In [ ]:
include = r"\bSFK\b|\bTWE\b|\bBAY\b|\bSSF\b|\bSBR\b|\bMIL\b|\bBWY\b|\bBUR\b|\bSMT\b|\bHPK\b|\bHIL\b|\bBEL\b|\bSCS\b|\bRWC\b|\bMPK\b|\bPAL\b|\bSTF\b|\bCAL\b|\bSAT\b|\bMVW\b|\bSUN\b|\bLAW\b|\bSCL\b|\bCPK\b|\bSJD\b|\bTAM\b|\bCAP\b|\bBHL\b|\bMHL\b|\bSMR\b|\bGIL\b"
exclude = r"@"
include_mask = tweets_2016_2019["station"].str.contains(include).fillna(False)
exclude_mask = tweets_2016_2019["station"].str.startswith(exclude).fillna(False)
tweets_2016_2019 = tweets_2016_2019[include_mask & ~exclude_mask]

In [ ]:
tweets_2016_2019.head()

Now that we're certain the station names are standardized, we'll organize our dataframe on station delays to have the number of delays reported at that station that year. 

In [ ]:
delays_per_station = tweets_2016_2019[["station", "clean_delay", "year"]].copy()
station_delays_groupby = delays_per_station.groupby(["station", "year"])
delays_per_station = station_delays_groupby.mean().reset_index()
delays_per_station

Now, we'll merge this dataframe with the one about the average weekly number of passengers on board at each station.

In [ ]:
passenger_merged = pd.merge(delays_per_station, filtered_passengers, on = ["station", "year"])
passenger_merged
passenger_merged.to_csv("passenger_and_delays_merged.csv")

Let's create a viz and see roughly how many passengers were affected by delays at each station from 2016-2019. 

In [ ]:
alt.Chart(passenger_merged).mark_bar().encode(
    x = "clean_delay",
    y = "station",
    color = "year"
).properties(title = "Average Station Delays on Caltrain", height = 500, width = 500)

It's strange that 8 stations don't have any tweeted train delays for some of the years throughout 2016-2019. This would be something to ask Caltrain about and also double check our dataset for. 

In [ ]:
alt.Chart(passenger_merged).mark_bar().encode(
    x = "year",
    y = "on_board",
).properties(title = "Average Number of Caltrain Riders 2016-2019", height = 400, width = 500)

It appears that the average number of on board passengers for Caltrain remained pretty consistent from 2016-2019. This shows that thousands of riders every week were affected by Caltrain delays, especially at 22nd Street station in San Francisco, where the most train delays were tweeted in this time period. 